<a href="https://colab.research.google.com/github/ezemartlopez/Proyectos/blob/main/Temperaturas_estaciones_meteorologicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def contar_numeros(cadena):
  datos = cadena.split()#divido la cadena y obtengo los valores
  numbers = [float(s) for s in datos if isfloat(s)]#obtengo los candidatos a float
  #es decir la fecha, temperatura_max y temperatura_min, cada uno en formato float
  return numbers[1:] #regreso solo apartir del indice 1 ya que tiene las temperaturas

def asignar_temperaturas(cadena):
  try:
    temperaturas = contar_numeros(cadena)
    max_temp = 0
    min_temp = 0
    if(len(temperaturas)==2):
      #significa que existen dos temperaturas 
      max_temp = temperaturas[0]
      min_temp = temperaturas[1]
    elif(len(temperaturas)==1):
      #como solo existe una temperatura, hay que decidir cual es max o min
      indice = cadena.find(str(temperaturas[0]))#busco el indice de donde aparece
      if(indice<=12):
        #significa que la temperatura es maxima
        max_temp = temperaturas[0]
        min_temp = None
      else:
        #sino es minima
        max_temp = None
        min_temp = temperaturas[0]
    else:
      #no hay temperaturas y se asignan nulas
      max_temp=None
      min_temp=None
    return (max_temp,min_temp)
  except ValueError:
    print(cadena)
    return None
def obtenerFecha(lista):
  return lista[0].lower()

def nombre_receptor(lista):
  #recibe una lista de elementos y agrupa los que son string
  nombre = [s for s in lista if s.isalpha()]
  return ' '.join(nombre)

def limpiar_cad(cadena):
  """La cadena es dificil de leer linea por linea, se debe considerar
  que cada linea tiene un formato complejo 'anio temp_max temp_min receptor'
  con espacios variados. pero devuelve un diccionario con los valores encotrados"""
  datos = cadena.split() # divido los elementos de la cadena en una lista: fecha, temp_min, temp_max, string,..., string
  nombre = nombre_receptor(datos) # obtengo el nombre del receptor temperatura
  fecha = obtenerFecha(datos) # obtengo la fecha de los datos
  max_temp,min_temp = asignar_temperaturas(cadena) # obtengo fecha temp_max y temp_min
  # creo el diccionario
  diccionario = {'fecha':fecha,'max_temp':max_temp,'min_temp':min_temp,'receptor':nombre}
  #print(diccionario)
  return diccionario

import zipfile
from urllib.request import urlopen
# from urllib import urlopen  # for python 2

import io
def obtenerInfoOnline():
  #url del archivo
  url = "https://ssl.smn.gob.ar/dpd/zipopendata.php?dato=regtemp"
  #obtengo el archivo zip del url
  archivo_zip = zipfile.ZipFile(io.BytesIO(urlopen(url).read()))
  #pero solo me intersa el siguiente archivo 'registro_temperatura365d_smn.txt'
  archivo = 'registro_temperatura365d_smn.txt'
  arch = archivo_zip.open(archivo,mode='r')
  data =[]
  for linea in arch.readlines():
    data.append(linea.decode('latin-1').replace('\r\n','').rstrip())
  return data[3:]


In [2]:
import pandas as pd
#Obtengo la info del clima online
lineas = obtenerInfoOnline()
datos = []
for linea in lineas:
  datos.append(limpiar_cad(linea))
interes = ['EL PALOMAR AERO','SAN FERNANDO AERO','AEROPARQUE AERO']#estaciones de recepcion de interes
df = pd.DataFrame(datos)
df['fecha'] = pd.to_datetime(df['fecha'], format = '%d%m%Y')

In [3]:
import plotly.express as px
df_aeroparque = df[df['receptor']=='AEROPARQUE AERO']
df_aeroparque=df_aeroparque.sort_values(by="fecha",ascending=True)
fig = px.scatter(df_aeroparque, x="fecha", y=['min_temp','max_temp'])
fig.show()

In [4]:
df_palomar=df[df['receptor']=='EL PALOMAR AERO']
df_palomar.sort_values(by='fecha')
fig = px.scatter(df_palomar, x="fecha", y=['min_temp','max_temp'])
fig.show()

In [5]:
df_sanfernando=df[df['receptor']=='SAN FERNANDO AERO']
df_sanfernando.sort_values(by='fecha')
fig = px.scatter(df_sanfernando, x="fecha", y=['min_temp','max_temp'])
fig.show()